In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install imutils

In [4]:
import os
import numpy as np
from imutils import paths
from tensorflow.keras.preprocessing.image import load_img
from keras.layers import UpSampling2D, Reshape, Activation, Conv2D, BatchNormalization, LeakyReLU, Input, Flatten, multiply
from keras.layers import Dense, Embedding, Conv2DTranspose
from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Dropout, Concatenate, GaussianNoise
import numpy as np
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [5]:
img_width, img_height = 128,128
img_channel = 3
img_shape = (img_width, img_height, img_channel)
num_classes = 20
z_dim = 2048

In [6]:
def build_generator():
    model = Sequential()
    model.add(Dense(2048, activation='relu', input_shape=(z_dim,)))
    model.add(Reshape((1,1,2048)))
    #model.add(UpSampling2D())
    model.add(Conv2DTranspose(128,kernel_size = 4))
    model.add(Activation('relu'))
    model.add(Conv2D(128, kernel_size = 4, padding = 'same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size = 4, padding = 'same'))
    model.add(BatchNormalization(momentum = 0.7))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(32, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum = 0.7))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(16, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum = 0.7))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(8, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum = 0.7))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(4, kernel_size = 3, padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(UpSampling2D())
    model.add(Conv2D(3,kernel_size = 3, padding ='same'))
    #model.add(LeakyReLU(alpha=0.2))
    #model.add(UpSampling2D())
    #model.add(Conv2D(3,kernel_size = 3, padding = 'same'))
    model.add(Activation('sigmoid'))
    
    z = Input(shape = (z_dim,))
    label = Input(shape=(1,), dtype = 'int32')
    
    label_embedding = Embedding(num_classes, z_dim, input_length =1)(label)
    label_embedding = Flatten()(label_embedding)
    joined = multiply([z,label_embedding])
    
    img = model(joined)
    return Model([z,label],img)

generator = build_generator()
generator.summary()

In [7]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(4, kernel_size = 3, input_shape = (128,128,6), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D())
    model.add(Conv2D(8, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D())
    model.add(Conv2D(16, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D())
    model.add(Conv2D(32, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D())
    model.add(Conv2D(64, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D())
    model.add(Conv2D(128, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D())
    model.add(Flatten())
    model.add(Dense(128))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    
    img = Input(shape= (img_shape))
    label = Input(shape= (1,), dtype = 'int32')
    
    label_embedding = Embedding(input_dim = num_classes, output_dim = np.prod(img_shape), input_length = 1)(label)
    label_embedding = Flatten()(label_embedding)
    label_embedding = Reshape(img_shape)(label_embedding)
    
    joined = Concatenate(axis=-1)([img, label_embedding])
    prediction = model(joined)
    return Model([img, label], prediction)

discriminator = build_discriminator()
discriminator.summary()

In [8]:
discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(0.0002, 0.5), metrics = ['accuracy'])

z = Input(shape=(z_dim,))
label = Input(shape= (1,))
img = generator([z,label])

discriminator.trainable = False
prediction = discriminator([img, label])

cgan = Model([z, label], prediction)
cgan.compile(loss= 'binary_crossentropy', optimizer = Adam(0.0002, 0.5))

In [9]:
def train(epochs, batch_size, save_interval):
    
    DIRECTORY = "../input/airplane-satellite-dataset/airplane-dataset-trans"
    CATEGORIES = ["A-10", "A-26" ,"B-1", "B-2", "B-29", "B-52", "Boeing", "C-5", "C-17", "C-21", "C-130", "C-135", "E-3", "F-16", "F-22", "KC-10", "P-63", "T-6", "T-43", "U-2"]
    Images = []
    Labels = []
    print("[INFO] loading images...")

    for category in CATEGORIES:
        path = os.path.join(DIRECTORY, category)
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            image = load_img(img_path, target_size=(128, 128), interpolation='nearest')
            temp = np.array(image.convert('RGB'), dtype='float32')
            Images.append(temp)
            Labels.append(category)
    labels_dic = dict([(y,x+1) for x,y in enumerate(sorted(set(Labels)))])
    num_Labels = []
    for x in Labels:
        num_Labels.append(labels_dic[x])
    Labels = np.array(num_Labels)
    Images = np.array(Images)
    
    Images = (Images - 127.5) / 127.5
    
    #Images = np.expand_dims(Images, axis=3)
    
    real = np.ones(shape=(batch_size, 1))
    fake = np.zeros(shape=(batch_size, 1))
    
    for iteration in range(epochs):
        
        idx = np.random.randint(0, Images.shape[0], batch_size)
        imgs, labels = Images[idx], Labels[idx]
        
        z = np.random.normal(0, 1, size=(batch_size, z_dim))
        gen_imgs = generator.predict([z, labels])
        
        d_loss_real = discriminator.train_on_batch([imgs,labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        z = np.random.normal(0, 1, size=(batch_size, z_dim))
        labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
        
        g_loss = cgan.train_on_batch([z, labels], real)
        
        if iteration % save_interval == 0:
            print('{} [D loss: {}, accuracy: {:.2f}] [G loss: {}]'.format(iteration, d_loss[0], 100 * d_loss[1], g_loss))
            save_image(iteration)

In [10]:
import matplotlib.pyplot as plt
def save_image(epoch):
    r, c = 4,5
    z = np.random.normal(0,1,(r*c, z_dim))
    labels = np.arange(0,20).reshape(-1,1)
    gen_image = generator.predict([z,labels])
    gen_image = 0.5 * gen_image + 0.5
    
    fig, axes = plt.subplots(r,c, figsize = (10,10))
    count = 0
    for i in range(r):
        for j in range(c):
            axes[i,j].imshow(gen_image[count,:,:,0],cmap = 'rgb')
            axes[i,j].axis('off')
            axes[i,j].set_title("Airplane: %d" % labels[count])
            count+=1
    plt.savefig('./cgan_%d.jpg' % epoch)
    plt.close()

In [ ]:
train(50000,64,500)